# **IMPORT LIBRARY**

In [2]:
import os
import sys
import time
import math
import glob
import sklearn
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsoluteError, MeanSquaredError, RootMeanSquaredError
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Dataset

In [63]:
df21 = pd.read_csv("SPKU Tahun 2021.csv")
df21.head()

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,202101,2021-01-22,DKI5 (Kebon Jeruk) Jakarta Barat,45,NaN,21,13,40,15,45,PM10,BAIK
1,202101,2021-01-23,DKI5 (Kebon Jeruk) Jakarta Barat,80,NaN,22,44,44,22,80,PM10,SEDANG
2,202101,2021-01-24,DKI5 (Kebon Jeruk) Jakarta Barat,27,NaN,14,9,29,---,29,CO,BAIK
3,202101,2021-01-25,DKI5 (Kebon Jeruk) Jakarta Barat,55,NaN,19,29,67,13,67,CO,SEDANG
4,202101,2021-01-26,DKI5 (Kebon Jeruk) Jakarta Barat,57,NaN,21,23,36,16,57,PM10,SEDANG


In [5]:
df21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1825 entries, 0 to 1824
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   periode_data  1825 non-null   int64 
 1   tanggal       1825 non-null   object
 2   stasiun       1825 non-null   object
 3   pm10          1825 non-null   object
 4   pm25          1763 non-null   object
 5   so2           1825 non-null   object
 6   co            1825 non-null   object
 7   o3            1825 non-null   object
 8   no2           1825 non-null   object
 9   max           1825 non-null   object
 10  critical      1809 non-null   object
 11  categori      1824 non-null   object
dtypes: int64(1), object(11)
memory usage: 171.2+ KB


In [6]:
df22 = pd.read_csv("SPKU Tahun 2022.csv")
df22.head()

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,202212,2022-12-31,DKI4,54,73,56,24,23,24,73,"PM2,5",SEDANG
1,202212,2022-12-30,DKI4,40,64,57,21,17,24,64,"PM2,5",SEDANG
2,202207,2022-07-06,DKI4,75,129,45,25,71,26,129,"PM2,5",TIDAK SEHAT
3,202207,2022-07-05,DKI4,66,110,47,16,61,23,110,"PM2,5",TIDAK SEHAT
4,202207,2022-07-04,DKI4,56,78,49,11,60,13,78,"PM2,5",SEDANG


In [7]:
df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   periode_data  365 non-null    int64 
 1   tanggal       365 non-null    object
 2   stasiun       365 non-null    object
 3   pm10          365 non-null    int64 
 4   pm25          365 non-null    int64 
 5   so2           365 non-null    int64 
 6   co            365 non-null    int64 
 7   o3            365 non-null    int64 
 8   no2           365 non-null    int64 
 9   max           365 non-null    int64 
 10  critical      364 non-null    object
 11  categori      365 non-null    object
dtypes: int64(8), object(4)
memory usage: 34.3+ KB


In [8]:
df23 = pd.read_csv("SPKU Tahun 2023.csv")
df23.head()

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,202302,2023-02-25,DKI5 Kebon Jeruk Jakarta Barat,35,-,13,12,31,18,35,PM10,BAIK
1,202302,2023-02-26,DKI5 Kebon Jeruk Jakarta Barat,23,-,14,9,32,11,32,O3,BAIK
2,202302,2023-02-27,DKI5 Kebon Jeruk Jakarta Barat,20,-,13,8,33,13,33,O3,BAIK
3,202302,2023-02-28,DKI5 Kebon Jeruk Jakarta Barat,30,-,21,11,28,18,30,PM10,BAIK
4,202303,2023-03-01,DKI1 Bunderan HI,38,44,50,8,19,27,50,3,BAIK


In [9]:
df24 = pd.read_csv("SPKU Tahun 2024.csv")
df24.head()

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,202401,2024-01-21,DKI3 Jagakarsa,51.0,65.0,45.0,9.0,8.0,79.0,79.0,NaN,SEDANG
1,202401,2024-01-22,DKI3 Jagakarsa,27.0,34.0,45.0,5.0,8.0,56.0,56.0,NaN,SEDANG
2,202401,2024-01-23,DKI3 Jagakarsa,NaN,52.0,46.0,6.0,9.0,51.0,52.0,PM25,SEDANG
3,202401,2024-01-24,DKI3 Jagakarsa,46.0,65.0,46.0,8.0,9.0,38.0,65.0,PM25,SEDANG
4,202401,2024-01-25,DKI3 Jagakarsa,37.0,55.0,47.0,7.0,11.0,28.0,55.0,PM25,SEDANG


In [10]:
df24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   periode_data  910 non-null    int64  
 1   tanggal       910 non-null    object 
 2   stasiun       910 non-null    object 
 3   pm10          896 non-null    float64
 4   pm25          902 non-null    float64
 5   so2           904 non-null    float64
 6   co            900 non-null    float64
 7   o3            905 non-null    float64
 8   no2           894 non-null    float64
 9   max           906 non-null    float64
 10  critical      886 non-null    object 
 11  categori      910 non-null    object 
dtypes: float64(7), int64(1), object(4)
memory usage: 85.4+ KB


# **DATA UNDERSTANDING**

In [42]:
# Import Data dan Pembuatan data Frame
joined_files = os.path.join("Dataset", "*.csv")

joined_list = glob.glob(joined_files)
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

In [43]:
# Pemahaman informasi data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   periode_data  4925 non-null   int64 
 1   tanggal       4925 non-null   object
 2   stasiun       4925 non-null   object
 3   pm10          4911 non-null   object
 4   pm25          4855 non-null   object
 5   so2           4919 non-null   object
 6   co            4915 non-null   object
 7   o3            4920 non-null   object
 8   no2           4909 non-null   object
 9   max           4921 non-null   object
 10  critical      4876 non-null   object
 11  categori      4924 non-null   object
dtypes: int64(1), object(11)
memory usage: 461.8+ KB


In [44]:
df.head()

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,202101,2021-01-22,DKI5 (Kebon Jeruk) Jakarta Barat,45,NaN,21,13,40,15,45,PM10,BAIK
1,202101,2021-01-23,DKI5 (Kebon Jeruk) Jakarta Barat,80,NaN,22,44,44,22,80,PM10,SEDANG
2,202101,2021-01-24,DKI5 (Kebon Jeruk) Jakarta Barat,27,NaN,14,9,29,---,29,CO,BAIK
3,202101,2021-01-25,DKI5 (Kebon Jeruk) Jakarta Barat,55,NaN,19,29,67,13,67,CO,SEDANG
4,202101,2021-01-26,DKI5 (Kebon Jeruk) Jakarta Barat,57,NaN,21,23,36,16,57,PM10,SEDANG


# **DATA PREPROCESSING**

**Menghilangkan data yang tidak diperlukan**

In [45]:
df = df.drop(['periode_data', 'critical', 'max', 'categori'], axis=1)

**Memeriksa Missing Value**

In [47]:
df = df.replace(['---'], np.NaN)
df = df.replace(['-'], np.NaN)
df = df.replace([' '], np.NaN)

In [48]:
df.fillna(value=np.nan)

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2
0,2021-01-22,DKI5 (Kebon Jeruk) Jakarta Barat,45,NaN,21,13,40,15
1,2021-01-23,DKI5 (Kebon Jeruk) Jakarta Barat,80,NaN,22,44,44,22
2,2021-01-24,DKI5 (Kebon Jeruk) Jakarta Barat,27,NaN,14,9,29,NaN
3,2021-01-25,DKI5 (Kebon Jeruk) Jakarta Barat,55,NaN,19,29,67,13
4,2021-01-26,DKI5 (Kebon Jeruk) Jakarta Barat,57,NaN,21,23,36,16
...,...,...,...,...,...,...,...,...
4920,2022-07-11,DKI4,66,112,47,15,84,29
4921,2022-07-10,DKI4,63,93,47,13,84,19
4922,2022-07-09,DKI2,61,93,48,13,107,25
4923,2022-07-08,DKI2,55,80,48,17,95,25


**Mengubah Tipe Data**

In [49]:
df['tanggal'] = pd.to_datetime(df['tanggal'])
df["pm10"] = df["pm10"].astype("float")
df["pm25"] = df["pm25"].astype("float")
df["so2"] = df["so2"].astype("float")
df["co"] = df["co"].astype("float")
df["o3"] = df["o3"].astype("float")
df["no2"] = df["no2"].astype("float")

In [50]:
df.dtypes

,0
tanggal,datetime64[ns]
stasiun,object
pm10,float64
pm25,float64
so2,float64
co,float64
o3,float64
no2,float64


**MENGATASI MISSING VALUE**

In [51]:
df.isnull().sum()

,0
tanggal,0
stasiun,0
pm10,304
pm25,403
so2,159
co,90
o3,102
no2,117


**Fill Missing Value dengan interpolasi**

In [53]:
df= df.interpolate(method='linear')

<ipython-input-53-1a96a489f374>:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df= df.interpolate(method='linear')


In [55]:
df['pm25'].bfill(inplace = True)

<ipython-input-55-06615e5dc7eb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pm25'].bfill(inplace = True)


In [56]:
df.isnull().sum()

,0
tanggal,0
stasiun,0
pm10,0
pm25,0
so2,0
co,0
o3,0
no2,0


**Mengatasi Inkonsisten data**

In [60]:
for i in range(df['stasiun'].size):
    if "DKI1" in df['stasiun'].values[i]:
      df.at[i, "stasiun"] = 'DKI1 Bunderan HI'
    if "DKI2" in df['stasiun'].values[i]:
      df.at[i, "stasiun"] = 'DKI2 Kelapa Gading'
    if "DKI3" in df['stasiun'].values[i]:
      df.at[i, "stasiun"] = 'DKI3 Jagakarsa'
    if "DKI4" in df['stasiun'].values[i]:
      df.at[i, "stasiun"] = 'DKI4 Lubang Buaya'
    if "DKI5" in df['stasiun'].values[i]:
      df.at[i, "stasiun"] = 'DKI5 Kebon Jeruk'

# **MEMISAHKAN DAN MENYIMPAN DATA BERDASARKAN STASIUN**

In [61]:
grouped = df.groupby('stasiun')

In [62]:
output_dir = 'Dataset_Stasiun'
os.makedirs(output_dir, exist_ok=True)

# Menyimpan setiap dataframe terpisah ke file CSV
for category, data in grouped:
    file_name = f'{output_dir}/df_{category}.csv'
    data.to_csv(file_name, index=False)
    print(f"File CSV untuk kategori {category} disimpan sebagai {file_name}")

File CSV untuk kategori DKI1 Bunderan HI disimpan sebagai Dataset_Stasiun/df_DKI1 Bunderan HI.csv
File CSV untuk kategori DKI2 Kelapa Gading disimpan sebagai Dataset_Stasiun/df_DKI2 Kelapa Gading.csv
File CSV untuk kategori DKI3 Jagakarsa disimpan sebagai Dataset_Stasiun/df_DKI3 Jagakarsa.csv
File CSV untuk kategori DKI4 Lubang Buaya disimpan sebagai Dataset_Stasiun/df_DKI4 Lubang Buaya.csv
File CSV untuk kategori DKI5 Kebon Jeruk disimpan sebagai Dataset_Stasiun/df_DKI5 Kebon Jeruk.csv
